# Lab 4: Image Classification

In this lab, we will be reviewing the most common classification method in remote sensing and learn how to leverage this Machine Learning (ML) technique with Google Earth Engine data. As per usual, we will be leveraging Dr. Qiusheng Wu's [Geemap](https://geemap.org/) package.

There are 4 Conceptual Questions, which will account for 40% of your lab grade. The 3 Challenge Questions will account for 60%.

# Requirements

Everytime you open a new notebook (or your kernel disconnects) you will need to run these requirements before you are able to produce any maps.

The 'pip install' lines are commented out becuase most everyone should already have these libraries installed on your devices, but in case you do not, uncomment and run the first two lines.

In [ ]:
# ! pip install geemap
# ! pip install pygis

import geemap
import ee
from geemap import basemaps

ee.Authenticate()
ee.Initialize(
    project="YOUR_PROJECT_NAME"
)  # Replace 'YOUR_PROJECT_NAME' with the name of your earth engine cloud project

# Graded Section

## Conceptual Questions (40%)

### Question 1 (10 Points)

What is our Random Forest model doing when we train it? What are the components required to do image classification?

### Question 2 (10 Points)

How does the spatial resolution of the input imagery (Landsat vs. Sentinel) potentially influence the accuracy of the classification results?

### Question 3 (10 Points)

Explain the importance of using relevant and representative training data for supervised classification. What are the potential consequences of using poor-quality or biased training data?

### Question 4 (10 Points)

Discuss the concept of a confusion matrix and how it is used to evaluate the accuracy of a classification model. What information can you derive from a confusion matrix?

## Challenge Questions (60%)

### Challenge 1 (20 Points)

Classify a Landsat image covering a **Country** of your choosing by training a Random Forest classifier on NLCD data, as we did in Part 1.

DATA:
- [Countries](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0) (FAO GAUL)
- [States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) (TIGER)
- [Landcover](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) (NLCD)
- [Imagery](https://developers.google.com/earth-engine/datasets/catalog/landsat) (Landsat, Surface Reflectance Tier 1)

TIP: The larger the country, the longer processing will take. Maybe stick to a small island country in the Caribbean?

### Challenge 2 (20 Points)

Classify a Sentinel image of a US state of your choosing using Random Forest and NLCD data.

DATA:
- [States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) (TIGER)
- [Landcover](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) (NLCD)
- [Imagery](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) (Sentinel-2, SR Harmonized)

### Challenge 3 (20 Points)

Use a confusion matrix to get the overall accuracy of your model for Challenge 2, as we did in Part 2 of this lab.

# Part 1: Classifying an Image

Random Forest classification is the most common method you will see in remote sensing ML applications. This model learns from our data by creating hundreds of decision trees (making a "forest") that test different patterns, training the model to find the most reliable metrics for prediction.

To run this classification model you need:
1. A multispectral image for your area.
2. Some vector file with georeferenced landcover class data.

### 1.1 Gather Training Data

Training data can be derived from field work or a predefined landcover database like the National Land Cover Database (NLCD), which we will be using in this lab. This only covers the United States, but we can apply our model to other geographies!

In [ ]:
# Create our map
rf = geemap.Map()
rf

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Define our region of interest
rf_region = ee.FeatureCollection("TIGER/2018/States").filter(
    ee.Filter.eq("NAME", "Texas")
)
rf.addLayer(rf_region, {}, "Texas")
rf.centerObject(rf_region, 6)

In [ ]:
# Load the Landsat 9 image collection
# This is the image we will be classifying
collection = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filter(ee.Filter.lt("CLOUD_COVER", 20))
    .filterDate("2016-01-01", "2016-12-31")
)


image = collection.median()


# Apply scaling factors to the image.
def apply_scale_factors(image):
    opticalBands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermalBands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)


dataset = apply_scale_factors(image)
dataset = dataset.clip(rf_region)

# Specify visualization parameters.
vis_natural = {
    "bands": ["SR_B4", "SR_B3", "SR_B2"],
    "min": 0.0,
    "max": 0.3,
}
vis_nir = {
    "bands": ["SR_B5", "SR_B4", "SR_B3"],
    "min": 0.0,
    "max": 0.3,
}

# Add data layers to the map.
rf.addLayer(dataset, vis_natural, "True color (432)")
rf.addLayer(dataset, vis_nir, "Color infrared (543)")

In [ ]:
# Get the data we will draw samples from
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(rf_region)
rf.addLayer(nlcd, {}, "NLCD")
rf.centerObject(nlcd)

In [ ]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": rf_region,
        "scale": 30,
        "numPixels": 1000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

rf.addLayer(points, {}, "training", False)

### 1.2 Train Classifier

In [ ]:
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]

# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = dataset.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a 10-tree random forest classifier from the training sample.
trained = ee.Classifier.smileRandomForest(10).train(
    features=training, classProperty=label, inputProperties=bands
)

### 1.3 Classify New Image

This next block may take a couple minutes to run - that's okay! Machine learning is computationaly intensive.

In [ ]:
# Mexico feature
mexico = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(
    ee.Filter.eq("ADM0_NAME", "Mexico")
)

landsat = apply_scale_factors(image)
lsat_mex = landsat.clip(mexico)

# Classify the image with the same bands used for training.
result = lsat_mex.select(bands).classify(trained)

# # Display the clusters with random colors.
rf.addLayer(result.randomVisualizer(), {}, "classified")

In [ ]:
rf

Map(bottom=53506.0, center=[32.04998888314202, -101.10297105589407], controls=(WidgetControl(options=['positio…

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps.

In [ ]:
# Pull NLCD Class Values
class_values = nlcd.get("landcover_class_values").getInfo()
print("Class Values: ", class_values)

# Pull NLCD Class Color Codes
class_palette = nlcd.get("landcover_class_palette").getInfo()
print("Class Pallette: ", class_palette)

Class Values:  [11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]
Class Pallette:  ['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [ ]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

rf.addLayer(landcover, {}, "Land cover")

In [ ]:
rf.add_legend(builtin_legend="NLCD")

## Sentinel Example

In [ ]:
# Create a new map for the Sentinel example
sentinel_map = geemap.Map()
sentinel_map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Define Region of Interest
sent_region = ee.FeatureCollection("TIGER/2018/States").filter(
    ee.Filter.eq("NAME", "Vermont")
)
sentinel_map.addLayer(sent_region, {}, "Vermont")
sentinel_map.centerObject(sent_region, 7)

In [ ]:
# Load Sentinel-2 image collection.
# Image we will classify
collection_sentinel = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filter(ee.Filter.lt("CLOUD_COVERAGE_ASSESSMENT", 10))
    .filterDate("2016-01-01", "2016-12-31")
)

# Select the desired bands and take the median composite

sentinel_image = collection_sentinel.select(["B2", "B3", "B4", "B8"]).median()

# Clip the image to the region
sentinel_dataset = sentinel_image.clip(sent_region)

# Specify visualization parameters for Sentinel-2
vis_natural_sentinel = {
    "bands": ["B4", "B3", "B2"],
    "min": 0.0,
    "max": 3000,  # Adjust max values based on Sentinel-2 data range (typically 0-10000)
}
vis_nir_sentinel = {
    "bands": ["B8", "B4", "B3"],
    "min": 0.0,
    "max": 3000,  # Adjust max values based on Sentinel-2 data range
}

# Add data layers to the map.
sentinel_map.addLayer(
    sentinel_dataset, vis_natural_sentinel, "Sentinel True color (432)"
)
sentinel_map.addLayer(
    sentinel_dataset, vis_nir_sentinel, "Sentinel Color infrared (843)"
)

# Load NLCD data for training as before
nlcd_sentinel = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(sent_region)
sentinel_map.addLayer(nlcd_sentinel, {}, "NLCD")
sentinel_map.centerObject(nlcd_sentinel)

In [ ]:
# Make the training dataset using NLCD and Sentinel data
# Use the Sentinel band names
bands_sentinel = ["B2", "B3", "B4", "B8"]
label_sentinel = "landcover"

# Sample the Sentinel image using the NLCD data
points_sentinel = nlcd_sentinel.sample(
    **{
        "region": sent_region,
        "scale": 30,  # NLCD resolution
        "numPixels": 1000,
        "seed": 0,
        "geometries": True,
    }
)

sentinel_map.addLayer(points_sentinel, {}, "training", False)

In [ ]:
# Overlay the points on the Sentinel imagery to get training data
training_sentinel = sentinel_dataset.select(bands_sentinel).sampleRegions(
    **{
        "collection": points_sentinel,
        "properties": [label_sentinel],
        "scale": 10,
    }  # Sentinel-2 resolution is 10m for these bands
)

# Train a Random Forest classifier
trained_sentinel = ee.Classifier.smileRandomForest(10).train(
    features=training_sentinel,
    classProperty=label_sentinel,
    inputProperties=bands_sentinel,
)

In [ ]:
# Classify the Sentinel image
classified_sentinel = sentinel_dataset.select(bands_sentinel).classify(trained_sentinel)

# Get NLCD class values and palette for visualization
class_values_sentinel = nlcd_sentinel.get("landcover_class_values").getInfo()
class_palette_sentinel = nlcd_sentinel.get("landcover_class_palette").getInfo()

# Set classification properties for visualization
classified_sentinel = classified_sentinel.set(
    "classification_class_values", class_values_sentinel
)
classified_sentinel = classified_sentinel.set(
    "classification_class_palette", class_palette_sentinel
)

# Add the classified image to the map
sentinel_map.addLayer(classified_sentinel, {}, "Sentinel Classified Land Cover")

# Add NLCD legend
sentinel_map.add_legend(builtin_legend="NLCD")

# Part 2: Evaluating Results

Just because we have a model that classifies an image does not mean it does so very well. We can get some metrics for evalutation. The most helpful of these is a [Confusion Matrix](https://www.numberanalytics.com/blog/confusion-matrix-remote-sensing-gis).

To get this information on our model, we need to 'test' it on new data that our model has not seen. If we test using our training data, we will always get 100% accuracy - which sounds great, but is not the truth.

## 2.1 Texas Model

Since we only ran our model on Mexico above and we have no validation dataset for Mexico, we need to run it real quick on Texas. Luckily, we already have the trained model, we just need to apply it to a Texas Landsat image.

In [ ]:
txVal = geemap.Map(basemap="HYBRID")
txVal

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Load the Landsat 9 image collection for Texas.
collection = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filter(ee.Filter.lt("CLOUD_COVER", 20))
    .filterDate("2016-01-01", "2016-12-31")
)

image = collection.median()


# Apply scaling factors to the image.
def apply_scale_factors(image):
    opticalBands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermalBands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)


# Define the Texas region (ensure rf_region is defined)
rf_region = ee.FeatureCollection("TIGER/2018/States").filter(
    ee.Filter.eq("NAME", "Texas")
)

dataset = apply_scale_factors(image)
dataset = dataset.clip(rf_region)

# Classify the Texas region using the trained model
texas_classified = dataset.select(bands).classify(trained)

# Get NLCD class values and palette for visualization (using the same as before for consistency)
class_values = nlcd.get("landcover_class_values").getInfo()
class_palette = nlcd.get("landcover_class_palette").getInfo()

# Set classification properties for visualization
texas_classified = texas_classified.set("classification_class_values", class_values)
texas_classified = texas_classified.set("classification_class_palette", class_palette)

# Add the classified Texas image to the map (assuming 'rf' map object exists)
txVal.addLayer(texas_classified, {}, "Texas Classified Land Cover")
txVal.centerObject(texas_classified)

In [ ]:
# Create validation points by sampling the NLCD data for the Texas region (assuming nlcd is defined)
validation_points = nlcd.sample(
    **{
        "region": rf_region,
        "scale": 30,
        "numPixels": 500,  # You can adjust the number of validation points
        "seed": 1,  # Use a different seed than the training points
        "geometries": True,
    }
)

# Overlay the validation points on the classified image to get predicted land cover values
validation_data = texas_classified.sampleRegions(
    **{"collection": validation_points, "properties": [label], "scale": 30}
)

# Calculate the confusion matrix
confusion_matrix = validation_data.errorMatrix(label, "classification")

# Print overall accuracy
print("Overall Accuracy:", confusion_matrix.accuracy().getInfo())

Overall Accuracy: 0.5081632653061224


## 2.2 Vermont Model

In [ ]:
# Evaluate the Vermont (Sentinel) model

# Create validation points by sampling the NLCD data for the Vermont region (assuming nlcd_sentinel is defined)
validation_points_sentinel = nlcd_sentinel.sample(
    **{
        "region": sent_region,
        "scale": 30,
        "numPixels": 500,  # Use the same number of validation points as for Texas
        "seed": 1,  # Use a different seed than the training points
        "geometries": True,
    }
)

# Overlay the validation points on the classified Sentinel image to get predicted land cover values (assuming classified_sentinel and label_sentinel are defined)
validation_data_sentinel = classified_sentinel.sampleRegions(
    **{
        "collection": validation_points_sentinel,
        "properties": [label_sentinel],
        "scale": 10,
    }  # Use Sentinel's resolution
)

# Calculate the confusion matrix
confusion_matrix_sentinel = validation_data_sentinel.errorMatrix(
    label_sentinel, "classification"
)

# Print overall accuracy
print("Overall Accuracy (Sentinel):", confusion_matrix_sentinel.accuracy().getInfo())

Overall Accuracy (Sentinel): 0.438
